## Load Required Packages and Data

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

from cytoolz import concatv

# NLTK for NLP utils and corpora
import nltk,pprint
from nltk import word_tokenize

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz

# Your code
import rnnlm; reload(rnnlm)
# import rnnlm_test; reload(rnnlm_test)

/home/huyue012/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[nltk_data] Downloading package treebank to
[nltk_data]     /home/huyue012/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


<module 'rnnlm' from '/home/huyue012/W266TextSummarization/rnnlm.py'>

### read nyt flat file

In [2]:
# need to remove quote?
# !tail -n 1000 data/nyt_structured_data.txt > data/nyt_test.txt
file = open('data/nyt_test.txt','rt')
read_array = file.readlines()
title=[]
lead=[]
body=[]
for line in read_array:
    data = line.split(' , ') #file id, headline, leading_paragraph and full_text
    title.append(data[1])
    lead.append(data[2])
    body.append(data[3])
file.close()
print(title[:2])
# print(body[2])

["'Baghdad Gallery Owner Hopes Culture Can Dispel Hate'", "'Sweet and Sour Sit Down to Dessert'"]


In [8]:
print(lead[1], len(lead[1]))

'RHUBARB is an alarmingly sour vegetable passed off as a fruit, but requiring a huge mound of sugar to effect the transformation.Crumb cake is a huge mound of sugar disguised as a cake, but demanding a bracing counterpoint -- say a swallow of coffee or tea -- to allay its cloying sweetness.' 292


### Tokenize and Sentence segmentation

In [4]:
# # tokenize built in load_data function
# from nltk import word_tokenize
# import nltk
# t = title[1]
# b = body[1]
# # print(word_tokenize(b))
# tokens = []
# body_tokens = []
# title_tokens = []
# for i in range(len(body)):
#     tokens.extend(nltk.wordpunct_tokenize(body[i]))
#     body_tokens.append(nltk.wordpunct_tokenize(body[i]))
#     title_tokens.append(nltk.wordpunct_tokenize(title[i]))
# # tokens = tokens[20:] #select tokens
# # text = nltk.Text(tokens)
# print(tokens[:20])

["'", 'He', 'may', 'be', 'the', 'last', 'hopeful', 'man', 'in', 'Iraq', '.', 'Amid', 'the', 'violence', ',', 'the', 'crumbling', 'economy', 'and', 'rising']


In [6]:
# # stem
# # do we need to do stemming?
# porter = nltk.PorterStemmer()
# lancaster = nltk.LancasterStemmer()
# [porter.stem(t) for t in tokens]
# # [lancaster.stem(t) for t in tokens]

In [22]:
# import sent_segment
# len(body_tokens)
# # body_tokens[999]

# sentences = []
# for t in range(len(body_tokens)):
#     sentence = np.array(sent_segment.segment_sentences(body_tokens[t]))
#     sentences.append(sentence)
# # sentence[:5]

### Further Preprocessing
leverage w266_common module
tokenize --> canonicalize digit --> canonicalize word --> vocabuluary --> split

In [3]:
# Helper libraries
# assuming body and title are in different arrays, but split training/test not random or shuffled

reload(utils)
V = 10000
# lower case, DG, Unknown, pad sentense start/end, split into training and test data sets
vocab, train_x_ids, test_x_ids, train_y_ids, test_y_ids = utils.load_data(lead, title, split=0.8, V=V, shuffle=42)

[nltk_data] Downloading package treebank to
[nltk_data]     /home/huyue012/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
Vocabulary: 10,000 types
Loaded 1,000 documents (4742 sentences)
Training set: 800 documents (3,920 sentences)
Test set: 200 documents (822 sentences)


In [6]:
print(len(lead),len(title))
print(len(train_x_ids),len(train_y_ids))
print(len(test_x_ids),len(test_y_ids))
print(len(train_x_ids[1]),len(train_x_ids[2]),train_x_ids[:2])
print(len(train_y_ids[0]),len(train_y_ids[2]),train_y_ids[:2])

1000 1000
800 800
200 200
300 832 [array([   0,    6,    0,  469,  283,    0,   56,    7, 1391,    0,  322,
        283,    0,  129,  469,  283,    0,  301,    7,   15,  129,    0,
        469,  443,   71,  283,  484,  868,  301,    0,   56,    7,  223,
          0,   77,  223,    0,   77,  297,    7, 3213,    0,    5,    0,
          7,   56,   77,  195,    0,  129,  469,  283,    0,  952,   77,
        443,  301,  283,  223,  206,  283,    0,    3,    0,  129,  469,
        283,    0,  206,  297,  868,   56,  322,  301,   77,  223,  529,
          0,  283,  206,  443,  223,  443,   56, 1391,    0,    7,  223,
        195,    0,  297,   77,   15,   77,  223,  529,    0,  297,  283,
        301,   77,  529,   77,  443,  868,   15,    0,    7,  223,  195,
          0,   71,  443,  301,   77,  129,   77,  206,    7,  301,    0,
         77,  223,  129,  443,  301,  283,  297,    7,  223,  206,  283,
          0,    3,    0,  469,    7,   15,    7,  223,    0,  223,    7,
         15,   1

## Embedding

### Sentence Embedding
For extractive modeling - sentence ranking

In [5]:
# # !pip install tensorflow_hub
# import tensorflow as tf
# import tensorflow_hub as hub
# embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
# embeddings = embed([
# "The quick brown fox jumps over the lazy dog.",
# "I am a sentence for which I would like to get its embedding"])
# session=tf.Session()
# session.run([tf.global_variables_initializer(), tf.tables_initializer()])
# print (tf.Session().run(embeddings))

## Build Core Graph

H : hidden state size = embedding size = per-cell output size

May need to update that

## Build Train Graph

## Trainint The Model

In [15]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation
        
        
    for i, (en_in, de_in, de_out) in enumerate(batch_iterator):
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.encoder_initial_h_, {self.embedding_encoder_: en_in})

        feed_dict = {
            self.embedding_encoder_: en_in,
            lm.decoder_outputs_: de_out,
            lm.decoder_inputs_: de_in,
            lm.encoder_initial_h_: h,
            lm.learning_rate_: learning_rate,
            lm.use_dropout_: use_dropout
        }
        ops = [loss, self.encoder_final_h_, train_op] # do i need self.encoder_final_h or decoder??  
   
        cost = 0.0
        vals = session.run(ops, feed_dict)
        cost = vals[0] #loss
        h = vals[1] #final_h

        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time


        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [16]:
def score_dataset(lm, session, x_ids, y_ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.rnnlm_batch_generator(x_ids,y_ids, batch_size=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=0.0, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

In [17]:
# Training parameters
# max_encoder_time = 25
# max_decoder_time = 25
batch_size = 50
learning_rate = 0.01 #default 0.01
num_epochs = 2

# Model parameters
model_params = dict(V=vocab.size, 
                    H=200, 
                    softmax_ns=200,
                    num_layers=1)

TF_SAVEDIR = "/tmp/w266/a3_model"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [40]:
# Will print status every this many seconds
import rnnlm; reload(rnnlm)
print_interval = 5

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.rnnlm_batch_generator(train_x_ids, train_y_ids, batch_size)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.    
        cost = run_epoch(lm, session, bi, 
                 learning_rate=learning_rate, train=True, 
                 verbose=True, tick_s=10)
        print("loss: {:.03f}  (perplexity: {:.02f})".format(cost, np.exp(cost)))

        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
#         if epoch == num_epochs:
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, train_ids, name="Train set")
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, test_ids, name="Test set")
        print("")

    
    # Save final model
    saver.save(session, trained_filename)

ValueError: not enough values to unpack (expected 2, got 1)

## Inference-WIP

In [ ]:
def sample_step(lm, session, input_w, initial_h):
    """Run a single RNN step and return sampled predictions.
  
    Args:
      lm : rnnlm.RNNLM
      session: tf.Session
      input_w : [batch_size] vector of indices
      initial_h : [batch_size, hidden_dims] initial state
    
    Returns:
      final_h : final hidden state, compatible with initial_h
      samples : [batch_size, 1] vector of indices
    """
    # Reshape input to column vector
    input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
    #### YOUR CODE HERE ####
    # Run sample ops
    samples = session.run(lm.pred_samples_, {lm.input_w_: input_w})
    final_h = session.run(lm.final_h_, {lm.input_w_: input_w})

        #### END(YOUR CODE) ####
        # Note indexing here: 
        #   [batch_size, max_time, 1] -> [batch_size, 1]
    return final_h, samples[:,-1,:]

In [ ]:
# Same as above, but as a batch
max_steps = 20
num_samples = 10
random_seed = 42

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(random_seed)
    
    # Load the trained model
    saver.restore(session, trained_filename)

    # Make initial state for a batch with batch_size = num_samples
    w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
    h = session.run(lm.initial_h_, {lm.input_w_: w})
    # We'll take one step for each sequence on each iteration 
    for i in range(max_steps):
        h, y = sample_step(lm, session, w[:,-1:], h)
        w = np.hstack((w,y))

    # Print generated sentences
    for row in w:
        for i, word_id in enumerate(row):
            print(vocab.id_to_word[word_id], end=" ")
            if (i != 0) and (word_id == vocab.START_ID):
                break
        print("")

## Score New Data - Placeholder

## Reference: NMT Tutorial
https://github.com/tensorflow/nmt

In [2]:
tf.__version__

'1.10.1'